In [ ]:
from transformers import T5Tokenizer,T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-small")
rl_model=T5ForConditionalGeneration.from_pretrained('./ppo_MB_sd_di20e_wtest/model')
rl_model.eval()

In [2]:

def optimize_one_text(u_model,one_text):
    inputs = tokenizer(one_text, return_tensors="pt")
    with torch.no_grad():
        outputs = u_model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=50,
        early_stopping=True)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

In [3]:
import pandas as pd
compare_promts = pd.read_parquet('blip2_ft_sd/result/other_generative_model_test.parquet')

In [ ]:
from tqdm import tqdm
rl_generated_caption=[]
generated_captions=compare_promts['generated_caption'].tolist()
len_generation=len(generated_captions)
for i in tqdm(range(len_generation)):
    rl_generated_caption.append(optimize_one_text(rl_model,generated_captions[i]))

In [ ]:
compare_promts['rl_generation']=rl_generated_caption

compare_promts.to_parquet(f'blip2_ft_sd/result/rl_other_generative_model_test.parquet')
compare_promts